In [1]:
from src.utils import load_data
import albumentations as A

img_data, label_data = load_data(reshape = False, scale_percent = 10, resize_only = True)

transform = A.Compose([
    A.Rotate(limit = 360, border_mode = 1, p = 1)
])

def augmentation(images, labels, transform, spr):
  # spr: samples per image
  new_images = []
  new_labels = []
  for i in range(len(images)):
    # se agrega la imagen original
    new_images.append(images[i])
    # se agregan "spr" rotaciones de la imagen original
    for _ in range(spr):
      transformed = transform(image = images[i])
      transformed_img = transformed["image"]
      new_images.append(transformed_img)
    # Se agregan los mismos spr + 1 labels (las imagenes rotadas y la original!)
    new_labels += [labels[i]] * (spr + 1)
  return new_images, new_labels

samples_per_image = 10
new_images, new_labels = augmentation(img_data, label_data, transform, samples_per_image)

Loading ../InsulatorsDataSet/03 Extraction/ images...
Loading ../InsulatorsDataSet/04 Extraction/ images...
Loading ../InsulatorsDataSet/05 Extraction/ images...
Loading ../InsulatorsDataSet/06 Extraction/ images...
Loading ../InsulatorsDataSet/07 Extraction/ images...
Loading ../InsulatorsDataSet/08 Extraction/ images...


In [4]:
np.random.seed(813)

def shuffle_along_axis(a, axis):
    idx = np.random.rand(*a.shape).argsort(axis = axis)
    return np.take_along_axis(a, idx, axis = axis)

def train_val_test_split(images, labels, N_VALIDATION, N_TRAIN, N_TEST):
    
    idx = shuffle_along_axis(np.arange(N_VALIDATION + N_TRAIN + N_TEST), 0)
    
    X_train = images[idx[:N_TRAIN]]
    y_train = labels[idx[:N_TRAIN]]

    X_val = images[idx[N_TRAIN:N_TRAIN+N_VALIDATION]]
    y_val = labels[idx[N_TRAIN:N_TRAIN+N_VALIDATION]]

    X_test = images[idx[N_TRAIN+N_VALIDATION:]]
    y_test = labels[idx[N_TRAIN+N_VALIDATION:]]
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [15]:
images = np.array(new_images) / 255.0 # normalize input

In [23]:
N_VALIDATION = 132
N_TRAIN = 1056
N_TEST = 132

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(images, np.array(new_labels), N_VALIDATION, N_TRAIN, N_TEST)

In [36]:
import torchvision.transforms.functional as transform
import tensorflow as tf

imageslist = []
for img in X_train[:64]:
    imageslist.append(transform.to_pil_image(transform.to_tensor(tf.image.resize(img, (227, 227)).numpy())))

In [27]:
from PIL import Image

In [37]:
def create_collage(width, height, listofimages):
    cols = 8
    rows = 8
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = listofimages
    i = 0
    x = 0
    y = 0
    for col in range(cols):
        for row in range(rows):
            print(i, x, y)
            new_im.paste(ims[i], (x, y))
            i += 1
            y += thumbnail_height
        x += thumbnail_width
        y = 0

    new_im.save("Collage.jpg")

create_collage(1600, 1600, imageslist)

0 0 0
1 0 200
2 0 400
3 0 600
4 0 800
5 0 1000
6 0 1200
7 0 1400
8 200 0
9 200 200
10 200 400
11 200 600
12 200 800
13 200 1000
14 200 1200
15 200 1400
16 400 0
17 400 200
18 400 400
19 400 600
20 400 800
21 400 1000
22 400 1200
23 400 1400
24 600 0
25 600 200
26 600 400
27 600 600
28 600 800
29 600 1000
30 600 1200
31 600 1400
32 800 0
33 800 200
34 800 400
35 800 600
36 800 800
37 800 1000
38 800 1200
39 800 1400
40 1000 0
41 1000 200
42 1000 400
43 1000 600
44 1000 800
45 1000 1000
46 1000 1200
47 1000 1400
48 1200 0
49 1200 200
50 1200 400
51 1200 600
52 1200 800
53 1200 1000
54 1200 1200
55 1200 1400
56 1400 0
57 1400 200
58 1400 400
59 1400 600
60 1400 800
61 1400 1000
62 1400 1200
63 1400 1400


In [30]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as transform
import torch.nn as nn

preparation = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [45]:
nnew_images = []
for n in new_images:
    temp = transform.to_tensor(n)
    nnew_images.append(preparation(transform.to_pil_image(temp)))

In [68]:
AlexNet_model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained = True)

Using cache found in C:\Users\danie/.cache\torch\hub\pytorch_vision_v0.6.0


In [72]:
AlexNet_model.classifier[4] = nn.Linear(4096, 1024)

In [73]:
AlexNet_model.classifier[6] = nn.Linear(1024, 10)

In [16]:
import torch.optim as optim
import torch.nn as nn

#Loss
criterion = nn.CrossEntropyLoss()

#Optimizer(SGD)
optimizer = optim.SGD(AlexNet_model.parameters(), lr=0.001, momentum=0.9)

In [17]:
#Instantiating CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Verifying CUDA
print(device)

#Move the input and AlexNet_model to GPU for speed if available
AlexNet_model.to(device)

cpu


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [71]:
nnew_images[0].shape

torch.Size([3, 224, 224])

In [59]:
import numpy as np

In [63]:
nnew_labels = torch.from_numpy(np.array(new_labels))

In [74]:
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for img, label in zip(nnew_images, nnew_labels):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = img.to(device), label.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        output = AlexNet_model(inputs)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training of AlexNet')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x36 and 9216x4096)